In [1]:
import ee
import geemap
import numpy as np
import time
# import geopandas as gpd

In [2]:
ee.Initialize()

In [3]:
#shp path
mazie ='maize.shp'
sugarcane = 'sugarcane.shp'
ot_cropty = 'others.shp'
noncrp = 'non-crop.shp'

#ee object
ee_mazie = geemap.shp_to_ee(mazie)
sugarc_ee = geemap.shp_to_ee(sugarcane)
ot_ee = geemap.shp_to_ee(ot_cropty)
np_ee = geemap.shp_to_ee(noncrp)
#feature collection
fc = ee.FeatureCollection([ee_mazie,sugarc_ee, ot_ee, np_ee])
fc = fc.flatten()
# fc = crop_ee.merge(noncrop_ee)
#define bounding box
bbox = ee.Geometry.bounds(fc)
aoi = bbox

In [4]:
composite = ee.Image('users/aadebowaledaniel/cpclasscomposite')

In [5]:
predbands = composite.bandNames()

In [6]:
#continue from here
trainingSample  = composite.sampleRegions(
    collection = fc,
    properties=['label'],
    scale = 10, 
    geometries = True
)
# trainingSample.size().getInfo()

rantraning = trainingSample.randomColumn('random')
split = 0.7
trainsample = rantraning.filter(ee.Filter.lt('random', split))
testsample = rantraning.filter(ee.Filter.gte('random', split))

#train random forest classifier
classifier = ee.Classifier.smileRandomForest(100).train(trainsample, 'label', predbands)


#confusion matrix train Accurracy
trainAccuracy = classifier.confusionMatrix()
print('Resubstitution error matrix: ', trainAccuracy.getInfo())
print('Training overall accuracy: ', trainAccuracy.accuracy().getInfo())

#classify features
testclassifier = testsample.classify(classifier)
#cconfusion matrix representing expected accuracy
testAccuracy = testclassifier.errorMatrix('label', 'classification')
print('confusion Matrix', testAccuracy.getInfo())
print('Valdation overall accuracy:', testAccuracy.accuracy().getInfo())

#classify image
Imageclassified = composite.classify(classifier)

Resubstitution error matrix:  [[737, 6, 10], [13, 568, 6], [1, 7, 961]]
Training overall accuracy:  0.9813772195755739
confusion Matrix [[179, 61, 38], [57, 150, 47], [16, 43, 371]]
Valdation overall accuracy: 0.7276507276507277


In [7]:
Map = geemap.Map(center=(0.9613,34.6536),zoom=8)
Map

Map(center=[0.9613, 34.6536], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [8]:
Map.addLayer(composite, {'bands': ['B4', 'B3', 'B2'], 'min': 0.0, 'max': 0.3, 'gamma': 2}, 'True colour composite')

In [9]:
sugarcanemask = Imageclassified.eq(2)

sugarcanemaskdd = sugarcanemask.updateMask(sugarcanemask)

sugarcanemask_vec = sugarcanemaskdd.reduceToVectors(
    geometryType= 'polygon',
    reducer= ee.Reducer.countEvery(),
    scale= 10,
    maxPixels= 1e13,
    geometry= aoi
)
Map.addLayer(sugarcanemask_vec, {'color':'green'}, 'Sugarcane')


In [10]:
taskvec = ee.batch.Export.table.toAsset(collection=sugarcanemask_vec,
description= 'Vegetation mask', assetId= 'users/aadebowaledaniel/vegetationmask')
taskvec.start()
#Monitor the task
while taskvec.status()['state'] in ['READY', 'RUNNING']:
  print(taskvec.status())
  time.sleep(5)
else:
  print(taskvec.status())

S', 'attempt': 1, 'id': 'ZXKUCZHD7VEVSFQSVA7YG3Y4', 'name': 'projects/earthengine-legacy/operations/ZXKUCZHD7VEVSFQSVA7YG3Y4'}
{'state': 'RUNNING', 'description': 'Vegetation mask', 'creation_timestamp_ms': 1620803009771, 'update_timestamp_ms': 1620803030301, 'start_timestamp_ms': 1620803030207, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'id': 'ZXKUCZHD7VEVSFQSVA7YG3Y4', 'name': 'projects/earthengine-legacy/operations/ZXKUCZHD7VEVSFQSVA7YG3Y4'}
{'state': 'RUNNING', 'description': 'Vegetation mask', 'creation_timestamp_ms': 1620803009771, 'update_timestamp_ms': 1620803030301, 'start_timestamp_ms': 1620803030207, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'id': 'ZXKUCZHD7VEVSFQSVA7YG3Y4', 'name': 'projects/earthengine-legacy/operations/ZXKUCZHD7VEVSFQSVA7YG3Y4'}
{'state': 'RUNNING', 'description': 'Vegetation mask', 'creation_timestamp_ms': 1620803009771, 'update_timestamp_ms': 1620803030301, 'start_timestamp_ms': 1620803030207, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'id': '

In [11]:
vis = {'min': 0, 'max': 3, 'pallete': ['black', 'green', 'yellow']}
Map.addLayer(Imageclassified, vis, 'Croptype')